# Inteligencia Artificial
## Practica 1: Agentes
### Sosa Romo Juan Mario - 320051926

Comenzamos importando el paquete y cumpliendo los requerimientos

In [ ]:
# Para poder usar los bots de telegram
!pip install python-telegram-bot

# Para poder usar cosas asincronas en colab
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

# Para poder dar resumenes de cosas
!pip install wikipedia

Importamos todo lo que vamos a usar y creamos los pares, ademas de un log

In [ ]:
import logging
import nltk
from nltk.chat.util import Chat, reflections
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes
import asyncio
import textwrap
import wikipedia
import random
import requests

wikipedia.set_lang("es")

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

selfName = "AI ChadBot Ultra v.69"
nombre = "amig@"

"""
Estos son pares de textos y sus respuestas, sirve para una IA tonta
"""
pairs = [

         ## Pares para conversaciones normales
    [
        r"(.*)(hola|ola)(.*)",
        [
            "Hola, {nombre} . ¿Cómo estás?",
            "Hola",
            "Hola {nombre}",
            "ola",
            "que oña",
            "q pedo mi {nombre}",
            "..."
        ]
    ],
    [
        r"(.*)(nombre es |me llamo )(.*)",
        [
            "¡Mucho gusto, {nombre}! ¿En qué puedo ayudarte?",
            "Abueno",
            "Hola {nombre}",
            "Muy bien, {nombre}  empecemos",
            "..."
        ]
    ],
    [
        r"(.*)(tonto|menso|bobo|burro|torpe|inútil|idiota|sonso|lento|tarado|bruto)(.*)",
        [
            "Vaya, eso no fue muy amable...",
            "Bueno, no todos podemos ser perfectos como tu. 🤖",
            "¿Te sientes mejor después de decir eso? Espero que sí. 😌",
            "Ser o no ser esa es la cuestion",
            "Bruh",
            "..."
        ]
    ],
    [
        r"(.*)(feo|horrible|desagradable|espantoso|asqueroso|ridículo)(.*)",
        [
            "Abueno",
            "¿Feo? Mas feo el SAT",
            "No todos podemos ser modelos de Instagram, aveces solo de Telgram.",
            "Hermano, soy un bot, estas perdiendo tu tiempo.",
            "Dime que tienes problemas de autoestima sin decirme que tienes problemas de autoestima. 😉",
            "..."
        ]
    ],
     [
        r"como estás(.*)",
        [
            "Bien *imita a tu ex*",
            "Bien, bien. *imita a tu jefe*",
            "Podría estar mejor si no tuviera que chambiar",
        ]
    ],
    [
        r"qué haces(.*)",
        [
            "Aquí, esperando que alguien me hable... como tú. 😏", # ala esto lo escribio gpt bro xd
            "Nada. *imita a tu ex*",
            "Contesto preguntas, mando memes y juzgo tus elecciones de vida en silencio. 😌",
            "Intentando no colapsar como tus tareas antes de la fecha límite.",
        ]
    ],
    [
        r"(.*)te amo(.*)",
        [
            "¡Ay, {nombre}! Esto es demasiado rápido. 😳",
            "Gracias",
            "Que original",
            "Uwu",
            "*Bloquear*",
            "..."
        ]
    ],
    [
        r"(.*)me gustas(.*)",
        [
            "Calmese",
            "Gracias pero no gracias",
            "*C sonroja*",
            "chido",
            "..."
        ]
    ],
    [
        r"(.*)te odio(.*)",
        [
            "Wow, {nombre}, qué agresivo. ¿Quieres un tiro?",
            "El odio consume mucha energía, mejor relájate y mándame un meme. 😌",
            "El hate es innecesario, {nombre}. Mejor dime un chiste. 😂",
            "*Bloquear*",
            "..."
        ]
    ],
    [
        r"(.*)dónde estás(.*)",
        [
            "En tu corazon",
            "Aquí",
            "En mi casa",
            "169.188.691.21"
        ]
    ],
    [
        r"(.*)(clima)(.*)",
        [
            "No se",
            "Creo que soleado",
            "Depende",
            "Preguntale a chat gpt",
            "...",
        ]
    ],
    [
        r"(.*)(sentido|motivo)(.*)",
        [
            "No se",
            "42",
            "Depende",
            "8",
            "Segun algunos expertos en fisica y filosofia (si, son investigadores en ambos campos) es como 69",
            "Preguntale a chat gpt"
        ]
    ],
    [
        r"(.*)(chiste)(.*)",
        [
            "Tu vida academica",
            "Tu vida amorosa",
            "No",
            "Dejame pensar ...",
            "Creo que 55",
            "Preguntale a chat gpt",
        ]
    ],


    ## si mencionan a otras IA

    [
        r"(.*)(como|eres|comparado con) (.*) (deepseek|gemini|chatgpt)(.*)",
        [
            "Soy único.",
            "Ah, *ps soy mejor",
            "Cada IA tiene sus fortalezas, menos gemini.",
            "Compararme con ChatGPT es *como comparar a un unicornio con un caballo*. Ambos tienen su encanto, ¿no?",
            "No los topo"
        ]
    ],
    [
        r"(.*)(cual|quien) es mejor (deepseek|gemini|chatgpt)(.*)",
        [
            "No se",
            "Preguntale a chat gpt",
            "Id win",
            "Soy como ese amigo raro"
        ]
    ],



    ## Respuestas dirigidas a mal uso de los comandos (escriben mal)

    [
        r"\\(memes|picc|buskar|inf|ayuda|startt|memesss|memez)",
        [
            "Esa no es la forma correcta, {nombre}. ¿Te falló el teclado? 🤨",
            "Casi lo logras, pero no.",
            "Creo que quisiste decir 'alt + f4'",
            "Comprate unas manos rey",
            "Los comandos no son tan difíciles, {nombre}.",
            "Skill issue",
            "No le sabes pa",
            "Regrese despues",
            "..."
        ]
    ],

    ## Si le preguntan como funciona o cosas complicadas
    [
        r"(.*)(algoritmo|funcionas)(.*)",
        [
            "No se",
            "Monton de ifs anidados"
        ]
    ],
    [
        r"(.*)(opinion)(.*)",
        [
            "No se",
            "Es complicado",
            "Tu que opinas?"
        ]
    ],
    [
        r"(.*)(recomienda|recomiendame|dame)(.*)",
        [
            "No",
            "A eso no le se",
            "ChatGPT.com"
        ]
    ],
    [
        r"(.*)(Que|Como|Donde|Por que)(.*)",
        [
            "Buena pregunta",
            "No lo habia pensado",
            "Eso es una pregunta?"
        ]
    ],

    [
        r"(.*)(Si|No|Ok|k|sip|nop)(.*)",
        [
            "Abueno",
            "Si",
            "No",
            "Okay",
            "Ok"
        ]
    ],

]

## Reddit auth

Aqui le doy todos los datos para que pueda darte memes, esta parte tiene secretos, plox no los uses mal.

In [ ]:
# datos de reddit para usar los memes
REDDIT_CLIENT_ID = ""
REDDIT_CLIENT_SECRET = ""

# esta cuenta la cree para esto no le des mal uso xd
REDDIT_USERNAME = ""
REDDIT_PASSWORD = ""

# 🌍 URL para obtener un token de acceso
TOKEN_URL = "https://www.reddit.com/api/v1/access_token"

# ⚡ Función para obtener un token de acceso a Reddit
def get_reddit_token():
    auth = requests.auth.HTTPBasicAuth(REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET)
    data = {"grant_type": "password", "username": REDDIT_USERNAME, "password": REDDIT_PASSWORD}
    headers = {"User-Agent": "meme-bot/0.1"}

    response = requests.post(TOKEN_URL, auth=auth, data=data, headers=headers)

    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        print("Error obteniendo el token:", response.text)
        return None

Esto nos crea las funcionesm comandos y como tal crea al chatbot

In [ ]:
# Crear el chatbot de NLTK
chatbot = Chat(pairs, reflections)

# Funciones asíncronas para los comandos
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Envía un mensaje de bienvenida al usuario.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    await update.message.reply_text(f"¡Hola! Soy tu Chadbot IA asíncrono {selfName}. Escribe algo para conversar.")

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Envía un mensaje con una lista de comandos disponibles.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    await update.message.reply_text("Comandos disponibles:\n/start\n/help\n/buscar\n/pic\n/info")

async def buscar(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Busca información en Wikipedia y envía un mensaje con el resumen.
    Si la búsqueda es ambigua, envía una lista de opciones.
    Si no se encuentra información, envía un mensaje de error.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    if not update.message:
        return

    query = ' '.join(context.args)

    if not query:
        await update.message.reply_text("Uso correcto: /buscar <término>")
        return

    await update.message.reply_text(f"Buscando información sobre: {query}...")

    try:
        resumen = wikipedia.summary(query, sentences=2)  # Obtener resumen corto
        url = f"https://es.wikipedia.org/wiki/{query.replace(' ', '_')}"
        mensaje = f"**{query}**\n{resumen}\n[Leer más]({url})"

        await update.message.reply_text(mensaje, parse_mode="Markdown")

    except wikipedia.exceptions.DisambiguationError as e:
        opciones = "\n".join(e.options[:5])  # Muestra hasta 5 opciones
        await update.message.reply_text(f"La búsqueda es ambigua. Prueba con:\n{opciones}")

    except wikipedia.exceptions.PageError:
        await update.message.reply_text("No encontré información sobre ese tema.")

    except Exception as e:
        await update.message.reply_text(f"Ocurrió un error: {e}")

async def pic(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Envía una imagen aleatoria de pantalla completa.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    image_url = f"https://picsum.photos/1920/1080?random={random.randint(1, 1000)}"

    await update.message.reply_photo(photo=image_url, caption="Aquí tienes un fondo de pantalla aleatorio!")

async def meme(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Obtiene un meme aleatorio de r/memes usando la API de Reddit con autenticación.
    """
    token = get_reddit_token()
    if not token:
        await update.message.reply_text("No pude autenticarme con Reddit")
        return

    headers = {"Authorization": f"bearer {token}", "User-Agent": "meme-bot/0.1"}

    subreddits = ["aigeneratedmemes", "aimemes", "ChatGPTMemes", "MemesMexico0","memes"]
    subreddit = random.choice(subreddits)

    url = f"https://oauth.reddit.com/r/{subreddit}/hot.json?limit=10"

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        await update.message.reply_text(f"Error {response.status_code}: No pude obtener memes")
        return

    data = response.json()
    posts = data.get("data", {}).get("children", [])

    if not posts:
        await update.message.reply_text("No encontré memes en este momento")
        return

    meme_post = random.choice(posts)["data"]
    meme_url = meme_post["url"]
    title = meme_post["title"]

    await update.message.reply_photo(photo=meme_url, caption=title)

async def info(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Envía información sobre el bot.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    await update.message.reply_text("Soy un Chatbot de '''Inteligencia Artificial''' implementado con NLTK y Telegram en modo asíncrono, soy capaz de darte imagenes para que uses de wallpaper y hacer busquedas muy tontas con Wikipedia, ademas te doy memes chidos de AI (generalmente).")

async def mensaje(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Maneja mensajes de texto del usuario.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    if not update.message:
        return
    user_message = update.message.text.lower()

    # Guardar el nombre si el usuario lo menciona
    if "mi nombre es" in user_message:
        nombre = user_message.split("mi nombre es")[-1].strip()
        context.user_data["nombre"] = nombre
    elif "nombre es" in user_message:
        nombre = user_message.split("nombre es")[-1].strip()
        context.user_data["nombre"] = nombre
    elif "me llamo" in user_message:
        nombre = user_message.split("me llamo")[-1].strip()
        context.user_data["nombre"] = nombre

    nombre = context.user_data.get("nombre", "amig@")

    respuesta = chatbot.respond(user_message)

    if respuesta:
        respuesta = respuesta.format(nombre=nombre)  # Reemplazamos el '{nombre}' por el nombre del usuario
        await update.message.reply_text(respuesta)
    else:
        await update.message.reply_text(f"Lo siento, {nombre}, no entendí lo que dijiste.")


async def error_handler(update: object, context: ContextTypes.DEFAULT_TYPE) -> None:
    """
    Maneja errores en el bot.

    Args:
      update: Objeto Update de Telegram.
      context: Objeto Context de Telegram.
    """
    logging.error(msg="Exception while handling an update:", exc_info=context.error)

async def main():
    """
    Inicia el bot en modo asíncrono.
    """
    TOKEN = ""  # Token de Telegram

    application = ApplicationBuilder().token(TOKEN).build()

    # Handlers de comandos
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("buscar", buscar))
    application.add_handler(CommandHandler("pic", pic))
    application.add_handler(CommandHandler("info", info))
    application.add_handler(CommandHandler("meme", meme))

    # Handler para no comandos
    application.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), mensaje))

    # Handler de errores
    application.add_error_handler(error_handler)

    await application.run_polling()

Ahora si iniciamos el bot y esta constantemente esperando que le digas cosas.

In [ ]:
if __name__ == '__main__':
    import asyncio
    asyncio.run(main())

RuntimeError: Cannot close a running event loop